# ME449 - Homework 3 - Zhengyang Kris Weng Submission

In [4]:
import numpy as np
import modern_robotics as mr
from tqdm import tqdm
import ur5_parameters
ur5 = ur5_parameters.UR5()

## Part0: Puppet function

## Part 1: Simulating a falling robot. 
In the first part, the robot will fall in gravity without
damping or the external spring (joint damping and spring stiffness are set to zero). Since there is
no damping or friction, the total energy of the robot (kinetic plus potential) should remain constant
during motion. Gravity is g = 9.81 m/s2 in the −ˆzs-direction, i.e., gravity acts downward.
Simulate the robot falling from rest at the home configuration for five seconds. The output data
should be saved as a .csv file, where each of the N rows has six numbers separated by commas.
This .csv file is suitable for animation with the CoppeliaSim UR5 csv animation scene. Adjust
the animation scene playback speed (“Time Multiplier”) so it takes roughly five seconds of wall
clock time to play your csv file. You can evaluate if your simulation is preserving total energy by
visually checking if the robot appears to swing to the same height (same potential energy) each
swing. Choose values of dt (a) where the energy appears nearly constant (without choosing dt
unnecessarily small) and (b) where the energy does not appear constant (because your timestep
is too coarse). Capture a video for each case and note the dt chosen for each case. Explain how
you would calculate the total energy of the robot at each timestep if you wanted to plot the total
energy to confirm that your simulation approximately preserves it.

In [5]:
# Puppet function:
def puppet_q1(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q1: free falling in gravity

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, np.zeros(n), g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [6]:
q1_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q1_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q1_thetamat, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.001, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat.csv', q1_thetamat, delimiter=',')

100%|██████████| 5000/5000 [00:32<00:00, 156.21it/s]


In [7]:
q1_thetamat_coarse, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat_coarse.csv', q1_thetamat_coarse, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.37it/s]


Loooking at video part1a and part1b.mp4, we can see that energy is not conserved due to error in numerical integration when timestep is too coarse. A good measure of total system energy would be the Hamiltonian of the system, as a sum of potential energy and kinetic energy of each link. By plotting the trend of Hamiltonian of each configuration in the trajectory, this will visualize the preservation of energy in the system. [TODO: IMPLEMENT COMPUTE H WHEN HAVE TIME]

## Part 2: Adding damping. 
Now experiment with different damping coefficients as the robot
falls from the home configuration. Damping causes a torque at each joint equal to the negative of
the joint rate times the damping. Create two videos showing that (a) when you choose damping to
be positive, the robot loses energy as it swings, and (b) when you choose damping to be negative,
the robot gains energy as it swings. Use t = 5 s and dt = 0.01 s, and for the case of positive
damping, the damping coefficient should almost (but not quite) bring the robot to rest by the end
of the video. Do you see any strange behavior in the simulation if you choose the damping constant
to be a large positive value? Can you explain it? How would this behavior change if you chose
shorter simulation timesteps?

In [8]:
def puppet_q2(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q2: Adding damping to robot. Damping causes a tau = - dtheta * damping

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        tau_damping = - damping * dthetalist
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [9]:
q2_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q2_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q2_thetamat, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 1.2, 0, 0)
# Save to csv file
np.savetxt('q2_thetamat.csv', q2_thetamat, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.30it/s]


In [10]:
q2_thetamat_neg, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, -0.02, 0, 0)
# Save to csv file
print(q2_thetamat_neg)
np.savetxt('q2_thetamat_neg.csv', q2_thetamat_neg, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 156.56it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [-3.07812861e-01 -9.92195181e+00  1.31700123e+01 -6.81968171e+00
  -8.36251555e-01  5.89264993e+01]
 [-3.02987560e-01 -1.00110250e+01  1.32499198e+01 -6.84260777e+00
  -8.76917115e-01  5.96387109e+01]
 [-2.97193787e-01 -1.00998345e+01  1.33216146e+01 -6.85772600e+00
  -9.17977817e-01  6.03581957e+01]]


In [11]:
q2_thetamat_large, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 20, 0, 0)
# Save to csv file
print(q2_thetamat_large)
np.savetxt('q2_thetamat_large.csv', q2_thetamat_large, delimiter=',')

  0%|          | 0/500 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/modern_robotics/core.py:927: RuntimeWarning: overflow encountered in multiply
  + np.dot(ad(Vi[:, i + 1]), Ai[:, i]) * dthetalist[i]
  3%|▎         | 17/500 [00:00<00:02, 165.73it/s]/usr/lib/python3/dist-packages/modern_robotics/core.py:143: RuntimeWarning: invalid value encountered in sin
  return np.eye(3) + np.sin(theta) * omgmat \
/usr/lib/python3/dist-packages/modern_robotics/core.py:144: RuntimeWarning: invalid value encountered in cos
  + (1 - np.cos(theta)) * np.dot(omgmat, omgmat)
/usr/lib/python3/dist-packages/modern_robotics/core.py:366: RuntimeWarning: invalid value encountered in multiply
  np.dot(np.eye(3) * theta \
/usr/lib/python3/dist-packages/modern_robotics/core.py:367: RuntimeWarning: invalid value encountered in cos
  + (1 - np.cos(theta)) * omgmat \
/usr/lib/python3/dist-packages/modern_robotics/core.py:368: RuntimeWarning: invalid value encountered in sin
  + (theta - np.sin(theta)) \
100%|████

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]]


As I increase the magnitude of damping coefficient in the simulation, the function start to run into numerical stability issues and starts to produce nan values in output. This happend because of the numerical instability in the euler integration and the fact that coarse timestep landing next iteration on gradients amplifying such effect; as a result simulation output grew at an extremely fast rate and overflowed. Increasing granularity in timestep helps addressing this issue.

## Part 3: Adding a spring. 
Make gravity and damping zero and design referencePos to return
a constant springPos at (0, 1, 1) in the {s} frame. The spring’s restLength is zero. Experiment
with different stiffness values, and simulate the robot for t = 10 s and dt = 0.01 s starting from
the home configuration. (a) Capture a video for a choice of stiffness that makes the robot oscillate
a couple of times and record the stiffness value. Considering the system’s total energy, does the
motion of the robot make sense? What do you expect to happen to the total energy over time?
Describe the strange behavior you see if you choose the spring constant to be large; if you don’t
see any strange behavior, explain why. (b) Now add a positive damping to the simulation that
makes the arm nearly come to rest by the end of the video. For both videos, record the stiffness
and damping you used.

In [69]:
def puppet_q3(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q3: Adding a static spring.

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        # Calculate damping
        print(f"Iteration {i}")
        tau_damping = - damping * dthetalist
        # Calculate spring force
        spring_force_vec = calculate_spring_wrench(thetalist, Slist, stiffness, restLength, referencePos_q3(i*dt))
        print(spring_force_vec)
        # Forward dynamics
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, spring_force_vec, Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

def referencePos_q3(t):
    """
    Generate a reference position for springPos 

    Args:
        t (float): current time (s)
    Returns:
        np.array: 3-vector of reference position
    """
    return np.array([0, 1, 1])

def calculate_spring_wrench(thetalist, Slist, stiffness, restLength, springPos):
    """
    Calculate the 6-vector spring wrench acting on the end-effector.

    Args:
        thetalist (np.array): n-vector of joint angles (rad)
        Mlist (np.array): 8 frames of link configuration at home pose
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
        springPOs (np.array): 3-vector of spring position in {s} frame
    Returns:
        np.array: 6-vector of spring forces and torque acting on the robot. Expressed in end-effector frame.
    """
    # Get end effector transformation matrix for current configuration

    eePos = mr.FKinSpace(ur5.M_EE, Slist, thetalist)
    print(f"eePos = {eePos}")
    # Extract position vector (first 3 elements of last column)
    p = np.array(eePos[:3,3])

    # Calculate spring length
    spring_length = np.linalg.norm(p - springPos) - restLength
    print(f"spring_length = {spring_length}")
    print(f"expected spring force = {stiffness * spring_length}")

    # Calculate spring force vector in {s} frame
    spring_force = stiffness * spring_length * (springPos - p) / np.linalg.norm(p - springPos)
    print(f"spring_force = {spring_force}")
    print(f"norm = {np.linalg.norm(spring_force)}")

    # Convert to end effector frame: T_{ee}^{s} * F_{s}
    spring_force_ee = mr.TransInv(eePos) @ np.array([*spring_force, 1]).T
    print(f"spring_force_ee = {spring_force_ee}")
    print(f"norm = {np.linalg.norm(spring_force_ee[:3])}")

    spring_wrench_ee = np.array([0, 0, 0, *spring_force_ee[:3]])
    return spring_wrench_ee

In [70]:
q3_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q3_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g_q3 = np.array([0, 0, 0])

q3_thetamat, _ = puppet_q3(q3_thetalist0, q3_dthetalist0, g_q3, ur5.Mlist, ur5.Slist, ur5.Glist, 10, 0.01, 0, 10, 0)
# Save to csv file
print(q3_thetamat)
np.savetxt('q3_thetamat.csv', q3_thetamat, delimiter=',')

  2%|▏         | 15/1000 [00:00<00:06, 145.27it/s]

Iteration 0
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 1
eePos = [[-1.        0.        0.        0.81725 ]
 [ 0.        0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]
spring_length = 1.5273057375918548
expected spring force = 15.273057375918548
spring_force = [-8.1725   8.0855  10.05491]
norm = 15.273057375918548
spring_force_ee = [ 8.98975  10.060401  7.89405   1.      ]
norm = 15.631484214424455
[ 0.        0.        0.        8.98975  10.060401  7.89405 ]
Iteration 2
eePos = [[

  4%|▍         | 45/1000 [00:00<00:06, 141.42it/s]

Iteration 29
eePos = [[-9.93095858e-01 -7.03114464e-04  1.17303545e-01  8.33499591e-01]
 [ 1.17305632e-01 -6.53504644e-03  9.93074359e-01  1.29633958e-01]
 [ 6.83391669e-05  9.99978399e-01  6.57240684e-03 -1.05945117e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6356568147034631
expected spring force = 16.356568147034633
spring_force = [-8.33499591  8.70366042 11.05945117]
norm = 16.356568147034633
spring_force_ee = [10.11173953 11.11556994  7.51253234  1.        ]
norm = 16.80003909474958
[ 0.          0.          0.         10.11173953 11.11556994  7.51253234]
Iteration 30
eePos = [[-9.92091147e-01 -8.59109741e-04  1.25516602e-01  8.33594876e-01]
 [ 1.25519515e-01 -7.45023172e-03  9.92063176e-01  1.25110334e-01]
 [ 8.28366312e-05  9.99971878e-01  7.49914398e-03 -1.12604581e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
spring_length = 1.6426202542261736
expected spring force = 16.426202542261738
spring_force = [-8

  8%|▊         | 75/1000 [00:00<00:06, 140.72it/s]

...

Iteration 942
eePos = [[ 0.28287887  0.93775118 -0.20150003  0.63556497]
 [ 0.06560876  0.19067074  0.97945912  0.37946871]
 [ 0.95690911 -0.29028845 -0.00758797  0.08392551]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2760072053773586
expected spring force = 12.760072053773586
spring_force = [-6.35564969  6.20531289  9.16074494]
norm = 12.760072053773587
spring_force_ee = [ 7.09025072 -8.08009766  7.04603153  1.        ]
norm = 12.853256153737215
[ 0.          0.          0.          7.09025072 -8.08009766  7.04603153]
Iteration 943
eePos = [[ 0.28759597  0.93592162 -0.2033206   0.63782802]
 [ 0.06639637  0.19229548  0.97908834  0.38260799]
 [ 0.95544758 -0.29508161 -0.00683839  0.09026728]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2710669144168965
expected spring force = 12.710669144168964
spring_force = [-6.37828019  6.17392014  9.09732719]
norm = 12.710669144168966
spring_force_ee = [ 6.9724912  -8.110702    7.03513174  1.      

 99%|█████████▉| 990/1000 [00:07<00:00, 135.89it/s]

Iteration 970
eePos = [[ 0.42023295  0.88847795 -0.18442127  0.70793208]
 [ 0.04569174  0.18226132  0.9821879   0.41971086]
 [ 0.90626516 -0.42117425  0.03599611  0.20582144]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.211867524185271
expected spring force = 12.11867524185271
spring_force = [-7.07932076  5.8028914   7.9417856 ]
norm = 12.11867524185271
spring_force_ee = [ 3.98434147 -9.19584571  7.00189444  1.        ]
norm = 12.225591234062083
[ 0.          0.          0.          3.98434147 -9.19584571  7.00189444]
Iteration 971
eePos = [[ 0.42528865  0.88661517 -0.1817776   0.71113343]
 [ 0.04335547  0.18065814  0.98258992  0.41917141]
 [ 0.90401873 -0.42576539  0.03839224  0.20788516]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.2126493786503043
expected spring force = 12.126493786503044
spring_force = [-7.1113343   5.80828585  7.92114837]
norm = 12.126493786503044
spring_force_ee = [ 3.87977524 -9.24597176  7.01336833  1.        ]

100%|██████████| 1000/1000 [00:07<00:00, 139.05it/s]

Iteration 998
eePos = [[ 0.55881493  0.82630639 -0.07031083  0.81861258]
 [-0.0509557   0.11883657  0.99160546  0.34231482]
 [ 0.82772543 -0.5505412   0.10851267  0.19733775]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.321719729905358
expected spring force = 13.21719729905358
spring_force = [-8.18612581  6.57685182  8.0266225 ]
norm = 13.21719729905358
spring_force_ee = [  1.13083061 -11.01012578   7.66490818   1.        ]
norm = 13.463003560684978
[  0.           0.           0.           1.13083061 -11.01012578
   7.66490818]
Iteration 999
eePos = [[ 0.56351635  0.82357784 -0.06456671  0.82308436]
 [-0.05549833  0.11572306  0.99172986  0.33684555]
 [ 0.82423859 -0.55527265  0.11091903  0.1943221 ]
 [ 0.          0.          0.          1.        ]]
spring_length = 1.3290442331999788
expected spring force = 13.290442331999788
spring_force = [-8.23084361  6.63154448  8.05677905]
norm = 13.290442331999788
spring_force_ee = [  1.02915868 -11.0939799    7.6993193